In [29]:
import pandas as pd
import gc

In [5]:
balance_pred = pd.read_csv("./some_balancing.csv")


In [6]:
dl_pred = pd.read_csv("./dl_support.csv")

In [12]:
dl_pred.head()

,click_id,is_attributed
0,0,0.000554
1,1,0.001969
2,2,0.000048
3,3,0.001204
4,4,0.000091


In [140]:
all_together = pd.concat([balance_pred, dl_pred], axis=1, sort=False)

In [14]:
all_together.head()

,click_id,is_attributed,click_id,is_attributed
0,0,0.438418,0,0.000554
1,1,0.438418,1,0.001969
2,2,0.008042,2,0.000048
3,3,0.008042,3,0.001204
4,4,0.008042,4,0.000091


In [15]:
from sklearn.linear_model import LogisticRegression

In [134]:
all_together.drop(all_together.columns[2], axis=1).head()

,is_attributed,is_attributed
0,0.438418,0.000554
1,0.438418,0.001969
2,0.008042,0.000048
3,0.008042,0.001204
4,0.008042,0.000091


predict LightGBM

In [18]:
dtypes = {
        'ip'            : 'uint32',
        'app'           : 'uint16',
        'device'        : 'uint16',
        'os'            : 'uint16',
        'channel'       : 'uint16',
        'is_attributed' : 'uint8',
        'click_id'      : 'uint32'
        }


In [64]:
lightGBM_tr = pd.read_csv("./train.csv", skiprows=range(1,144903891), nrows=40000000,dtype=dtypes, usecols=['ip','app','device','os', 'channel', 'click_time', 'is_attributed'])


In [23]:
import lightgbm as lgb

In [24]:
bst = lgb.Booster(model_file='lgbm.txt')

In [65]:
test = lightGBM_tr

In [26]:
def new_feats_light(df):
    train_df = df.copy()
    print('Extracting new features...')
    train_df['hour'] = pd.to_datetime(train_df.click_time).dt.hour.astype('uint8')
    train_df['day'] = pd.to_datetime(train_df.click_time).dt.day.astype('uint8')

    gc.collect()

    print('grouping by ip-day-hour combination...')
    gp = train_df[['ip','day','hour','channel']].groupby(by=['ip','day','hour'])[['channel']].count().reset_index().rename(index=str, columns={'channel': 'ip_tcount'})
    train_df = train_df.merge(gp, on=['ip','day','hour'], how='left')
    del gp
    gc.collect()

    print('grouping by ip-app combination...')
    gp = train_df[['ip', 'app', 'channel']].groupby(by=['ip', 'app'])[['channel']].count().reset_index().rename(index=str, columns={'channel': 'ip_app_count'})
    train_df = train_df.merge(gp, on=['ip','app'], how='left')
    del gp
    gc.collect()


    print('grouping by ip-app-os combination...')
    gp = train_df[['ip','app', 'os', 'channel']].groupby(by=['ip', 'app', 'os'])[['channel']].count().reset_index().rename(index=str, columns={'channel': 'ip_app_os_count'})
    train_df = train_df.merge(gp, on=['ip','app', 'os'], how='left')
    del gp
    gc.collect()


    # Adding features with var and mean hour (inspired from nuhsikander's script)
    print('grouping by : ip_day_chl_var_hour')
    gp = train_df[['ip','day','hour','channel']].groupby(by=['ip','day','channel'])[['hour']].var().reset_index().rename(index=str, columns={'hour': 'ip_tchan_count'})
    train_df = train_df.merge(gp, on=['ip','day','channel'], how='left')
    del gp
    gc.collect()

    print('grouping by : ip_app_os_var_hour')
    gp = train_df[['ip','app', 'os', 'hour']].groupby(by=['ip', 'app', 'os'])[['hour']].var().reset_index().rename(index=str, columns={'hour': 'ip_app_os_var'})
    train_df = train_df.merge(gp, on=['ip','app', 'os'], how='left')
    del gp
    gc.collect()

    print('grouping by : ip_app_channel_var_day')
    gp = train_df[['ip','app', 'channel', 'day']].groupby(by=['ip', 'app', 'channel'])[['day']].var().reset_index().rename(index=str, columns={'day': 'ip_app_channel_var_day'})
    train_df = train_df.merge(gp, on=['ip','app', 'channel'], how='left')
    del gp
    gc.collect()

    print('grouping by : ip_app_chl_mean_hour')
    gp = train_df[['ip','app', 'channel','hour']].groupby(by=['ip', 'app', 'channel'])[['hour']].mean().reset_index().rename(index=str, columns={'hour': 'ip_app_channel_mean_hour'})
    print("merging...")
    train_df = train_df.merge(gp, on=['ip','app', 'channel'], how='left')
    del gp
    gc.collect()

    print("vars and data type: ")
    train_df.info()
    train_df['ip_tcount'] = train_df['ip_tcount'].astype('uint16')
    train_df['ip_app_count'] = train_df['ip_app_count'].astype('uint16')
    train_df['ip_app_os_count'] = train_df['ip_app_os_count'].astype('uint16')
    return train_df

In [66]:
test_lightgbm = new_feats_light(test)

Extracting new features...
grouping by ip-day-hour combination...
grouping by ip-app combination...
grouping by ip-app-os combination...
grouping by : ip_day_chl_var_hour
grouping by : ip_app_os_var_hour
grouping by : ip_app_channel_var_day
grouping by : ip_app_chl_mean_hour
merging...
vars and data type: 
<class 'pandas.core.frame.DataFrame'>
Int64Index: 40000000 entries, 0 to 39999999
Data columns (total 16 columns):
ip                          uint32
app                         uint16
device                      uint16
os                          uint16
channel                     uint16
click_time                  object
is_attributed               uint8
hour                        uint8
day                         uint8
ip_tcount                   int64
ip_app_count                int64
ip_app_os_count             int64
ip_tchan_count              float64
ip_app_os_var               float64
ip_app_channel_var_day      float64
ip_app_channel_mean_hour    float64
dtypes: float64(4),

In [37]:
def new_feats_nn(df):
    train_df = df.copy()
    print('hour, day, wday....')
    train_df['hour'] = pd.to_datetime(train_df.click_time).dt.hour.astype('uint8')
    train_df['day'] = pd.to_datetime(train_df.click_time).dt.day.astype('uint8')
    train_df['wday']  = pd.to_datetime(train_df.click_time).dt.dayofweek.astype('uint8')
    print('grouping by ip-day-hour combination....')
    gp = train_df[['ip','day','hour','channel']].groupby(by=['ip','day','hour'])[['channel']].count().reset_index().rename(index=str, columns={'channel': 'qty'})
    train_df = train_df.merge(gp, on=['ip','day','hour'], how='left')
    del gp; gc.collect()
    print('group by ip-app combination....')
    gp = train_df[['ip','app', 'channel']].groupby(by=['ip', 'app'])[['channel']].count().reset_index().rename(index=str, columns={'channel': 'ip_app_count'})
    train_df = train_df.merge(gp, on=['ip','app'], how='left')
    del gp; gc.collect()
    print('group by ip-app-os combination....')
    gp = train_df[['ip','app', 'os', 'channel']].groupby(by=['ip', 'app', 'os'])[['channel']].count().reset_index().rename(index=str, columns={'channel': 'ip_app_os_count'})
    train_df = train_df.merge(gp, on=['ip','app', 'os'], how='left')
    del gp; gc.collect()
    print("vars and data type....")
    train_df['qty'] = train_df['qty'].astype('uint16')
    train_df['ip_app_count'] = train_df['ip_app_count'].astype('uint16')
    train_df['ip_app_os_count'] = train_df['ip_app_os_count'].astype('uint16')
    print("label encoding....")
    from sklearn.preprocessing import LabelEncoder
    train_df[['app','device','os', 'channel', 'hour', 'day', 'wday']].apply(LabelEncoder().fit_transform)
    print ('final part of preparation....')
    train_df.drop(['click_time','ip'],1,inplace=True)
    return train_df

In [67]:
test_nn = new_feats_nn(test)

hour, day, wday....
grouping by ip-day-hour combination....
group by ip-app combination....
group by ip-app-os combination....
vars and data type....
label encoding....
final part of preparation....


In [42]:
test_nn.head()

,app,device,os,channel,is_attributed,hour,day,wday,qty,ip_app_count,ip_app_os_count
0,3,1,13,379,0,14,6,0,1,1650,420
1,3,1,19,379,0,14,6,0,1,1369,350
2,3,1,13,379,0,14,6,0,1,334,83
3,14,1,13,478,0,14,6,0,1,2382,508
4,3,1,13,379,0,14,6,0,1,104,35


In [39]:
from keras.models import load_model

In [40]:
model_nn = load_model("nn.h5")

In [68]:
def get_keras_data(dataset):
    X = {
        'app': np.array(dataset.app),
        'ch': np.array(dataset.channel),
        'dev': np.array(dataset.device),
        'os': np.array(dataset.os),
        'h': np.array(dataset.hour),
        'd': np.array(dataset.day),
        'wd': np.array(dataset.wday),
        'qty': np.array(dataset.qty),
        'c1': np.array(dataset.ip_app_count),
        'c2': np.array(dataset.ip_app_os_count)
    }
    return X

In [51]:
predictors = ['app','device','os', 'channel', 'hour', 'day', 
              'ip_tcount', 'ip_tchan_count', 'ip_app_count',
              'ip_app_os_count', 'ip_app_os_var',
              'ip_app_channel_var_day','ip_app_channel_mean_hour']

In [70]:
sub = pd.DataFrame()
# sub['click_id'] = test_df['click_id'].astype('int')

In [71]:
sub['is_attributed'] = bst.predict(test_lightgbm[predictors])

In [72]:
pred_light = sub

In [73]:
import numpy as np

In [74]:
test_nn_x = get_keras_data(test_nn)

In [88]:
test_nn_y = test_nn["is_attributed"].values

In [76]:
pred_nn = model_nn.predict(test_nn_x, batch_size=20000, verbose=1)

40000000/40000000 [==============================] - 543s 14us/step


In [80]:
pred_light = pred_light.values

In [82]:
together = np.hstack([pred_light, pred_nn])

In [86]:
len_train = int(len(together) - len(together) * 0.1)

In [89]:
together_tr = together[:len_train]
together_val = together[len_train:]

In [90]:
labels_tr = test_nn_y[:len_train]
labels_val = test_nn_y[len_train:]

In [127]:
regr = LogisticRegression(C=1, n_jobs=12, solver="saga")

In [92]:
from sklearn.metrics import roc_auc_score

In [128]:
regr.fit(together, test_nn_y)

LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=12,
          penalty='l2', random_state=None, solver='saga', tol=0.0001,
          verbose=0, warm_start=False)

In [129]:
pred_log = np.min(regr.predict_proba(together_val), axis=1)

In [130]:
roc_auc_score(labels_val, pred_log)

0.9520247942087371

In [135]:
all_together = all_together.drop(all_together.columns[2], axis=1)

In [136]:
all_together.values

array([[4.38418195e-01, 5.53669500e-04],
       [4.38418195e-01, 1.96920030e-03],
       [8.04238165e-03, 4.75976920e-05],
       ...,
       [8.04238165e-03, 1.76672340e-04],
       [1.00000000e+00, 1.89782950e-03],
       [8.04238165e-03, 1.84954480e-06]])

In [132]:
all_together.values

array([[0.00000000e+00, 4.38418195e-01, 0.00000000e+00, 5.53669500e-04],
       [1.00000000e+00, 4.38418195e-01, 1.00000000e+00, 1.96920030e-03],
       [2.00000000e+00, 8.04238165e-03, 2.00000000e+00, 4.75976920e-05],
       ...,
       [1.87904670e+07, 8.04238165e-03, 1.87904670e+07, 1.76672340e-04],
       [1.87904660e+07, 1.00000000e+00, 1.87904660e+07, 1.89782950e-03],
       [1.87904680e+07, 8.04238165e-03, 1.87904680e+07, 1.84954480e-06]])

In [138]:
pred = np.min(regr.predict_proba(all_together.values), axis=1)

In [125]:
all_together["click_id"].values[:, 0]

array([       0,        1,        2, ..., 18790467, 18790466, 18790468])

In [141]:
sub = pd.DataFrame()
sub['click_id'] = all_together["click_id"].values[:, 0].astype('int')

print("predicting....")
sub['is_attributed'] = pred
# del test_df; gc.collect()
print("writing....")
sub.to_csv('stacking2.csv',index=False)

predicting....
writing....
